# Modelo Matemático - Problema Fake News

## Variáveis

- $x_{i,u} \in \{0,1\}$: vale 1 se o recurso $i$ foi alocado ao servidor $u$; 0 caso contrário;
- $y_{u,f} \in \{0,1\}$: vale 1 se o servidor $u$ foi atingido pela fake news em tempo menor ou igual a T, no contexto em que a fake news partiu do servidor $f$; 0 caso contrário;
- $t_{v,f} \in \mathbb{Z}$: tempo mínimo para o servidor $v$ ser atingido pela fake news, no contexto em que essa fake news partiu do servidor $f$;
- $S$: a maior quantidade de servidores atingidos pela fake news olhando o contexto geral em que ela partiu de qualquer vértice

## Função Objetivo

A fake news pode partir de qualquer vértice pertencente ao conjunto de vértices $V$ da instância. Desse modo, ao analisar o cenário, podemos obter alcancabilidades de servidores diferentes a depender da fonte $s$ a partir da qual partiu a fake news. Para podermos distribuir os recursos, temos que analisar o pior caso, ou seja, o vértice $s$ (fonte da fake news) que provoca maior alcançabilidade de vértices e, com isso, nosso objetivo é minimizar essa alcançabilidade do pior caso. Portanto, a função objetivo é minimizar a maior quantidade de servidores atingidos pela fake news, considerando o pior caso de $s$:

$$
\min S
$$

## Restrições

- Determinar o valor de S para cada fonte $s$ possível como a quantidade de servidores atingidos pela fake news no contexto em que ela partiu de $s$:
$$
S \geq \sum_{u \in V}y_{u,s} \space \forall s \in V
$$    
    
- O tempo em que um vértice $v$ é alcançado pela fake news é menor ou igual a $T$. Essa restrição força $y_{v,s} = 1$ se o tempo $t_{v,s} \lt T$. Vale ressaltar que é uma restrição "perigosa", pois ela não garante $y_{v,s} = 1$ quando $t_{v,s} = T$ nem $y_{v,s} = 0$ quando $t_{v,s} \gt T$:
$$
t_{v,s} \geq (T+1)(1-y_{v,s}) \space \forall v,s \in V
$$

- O tempo em que um vértice $v$ é alcançado pela fake news é calculado pelo tempo em que seu predecessor $u$ é alcançado somado ao tempo gasto no enlance, bem como a um valor de $\delta$ se algum dos recursos foi alocado no vértice $u$:
$$
t_{v,s} \leq t_{u,s} + t_{uv} + \delta*\sum_{i=1}^{\alpha}x_{i,u}, \space \forall (u,v) \in E
$$

- Para cada vértice, só pode ter no máximo um dos recursos alocados nele:
$$
\sum_{i=1}^{\alpha}x_{i,u} \leq 1, \space \forall u \in V
$$

- Para cada recurso, só pode ter no máximo um dos vértices com ele alocado:
$$
\sum_{u \in V}x_{i,u} \leq 1, \space \forall i=1,..,\alpha
$$

- Um recurso $i$ só pode ser alocado a um servidor $u$ se o tempo em que esse servidor $u$ foi alcançado for superior ao instante $\beta_{i}$ em que o recurso $i$ fica disponível:
$$
t_{u,s} \geq \sum_{i=1}^{\alpha}x_{i,u}*\beta_{i}, \space \forall u,s \in V
$$

- No contexto em que a fake news parte do servidor $s$, o valor de $y$ para o vértice $s$ é igual a 1 e o tempo em que o vértice $s$ é alcançado é 0:
$$
y_{s,s} = 1
$$
$$
t_{s,s} = 0
$$
$$
\forall s \in V
$$

In [ ]:
from collections import defaultdict
from gurobipy import GRB, Model

def lerArquivoDat(nomeArquivo):
    with open(nomeArquivo, "r") as file:
        # Ler a primeira linha
        linha1 = file.readline().strip().split()
        n = int(linha1[0])        # Tamanho da grade n x n
        delta = int(linha1[1])    # Tempo adicional δ
        T = int(linha1[2])        # Tempo limite T
        
        # Ler a segunda linha
        m = int(file.readline().strip())  # Número de grupos de recursos
        
        # Ler as próximas m linhas (recursos disponíveis)
        betas = {}
        for i in range(m):
            t, k = map(int, file.readline().strip().split())
            for j in range(1, k+1):
                betas[j+k*i] = t
        alfa = len(betas)
        
        # Ler os arcos
        arcos = []
        vertices = set({})
        for linha in file:
            u, v, t = linha.strip().split()
            arcos.append((u,v, int(t)))
            vertices.add(u)
            vertices.add(v)
        vertices = list(sorted(vertices))

    return {
        "n": n,
        "delta": delta,
        "T": T,
        "alfa": alfa,
        "betas": betas,
        "arcos": arcos,
        "vertices": vertices
    }

def main(nomeArquivo):
    dados = lerArquivoDat(nomeArquivo)

    modelo = Model("fake_news")
    
    vertices = dados["vertices"]
    
    x = {}
    for i in range(1,dados["alfa"]+1):
      x[i] = {}
      for u in vertices:
          x[i][u] = modelo.addVar(vtype=GRB.BINARY, name=f"x_{i}_{u}")
    
    y = {}
    for u in vertices:
      y[u] = {}
      for f in vertices:  
          y[u][f] = modelo.addVar(vtype=GRB.BINARY, name=f"y_{u}_{f}")
    
    t = {}
    for v in vertices:
      t[v] = {}
      for f in vertices:  
          t[v][f] = modelo.addVar(vtype=GRB.INTEGER, name=f"t_{v}_{f}")
    
    S = modelo.addVar(vtype=GRB.INTEGER, name=f"S")
    
    
    modelo.setObjective(S, GRB.MINIMIZE)
    
    for s in vertices:
        modelo.addConstr(S >= sum(y[u][s] for u in vertices))
    
    
    for s in vertices:
        for v in vertices:
            modelo.addConstr(t[v][s] >= (dados["T"]+1)*(1-y[v][s]))
        
    for s in vertices:
        for (u,v,tuv) in dados["arcos"]:
               modelo.addConstr(t[v][s] <= t[u][s] + tuv + dados["delta"]*sum(x[i][u] for i in range(1, dados["alfa"]+1)), name=f"cálculo de t_{v}")
    
    for u in vertices:
            modelo.addConstr(sum(x[i][u] for i in range(1,dados["alfa"]+1)) <= 1)
    
    for i in range(1,dados["alfa"]+1):
            modelo.addConstr(sum(x[i][u] for u in vertices) <= 1)
    
    for s in vertices:
        for u in vertices:
                modelo.addConstr(t[u][s] >= sum(x[i][u]*dados["betas"][i] for i in range(1,dados["alfa"]+1)))
    
    for s in vertices:
        modelo.addConstr(y[s][s] == 1)
        modelo.addConstr(t[s][s] == 0)
    
    
    modelo.optimize()
    
    if(modelo.status==GRB.OPTIMAL):
      print(f"\n\nFunção Objetivo: {modelo.objVal}")
      for f in vertices:
          print(f"FONTE DA FAKE NEWS: {f}")
          for i in vertices:
              print(f"\tVertice {i}: {y[i][f].X}")
              if(y[i][f].X==1):
                  print(f"\t\tTempo: {t[i][f].X}")
      for i in range(1,dados["alfa"]+1):
          for u in vertices:
              if(x[i][u].X==1):
                  print(f"Recurso {i} aplicado no vertice {u}? {x[i][u].X}")
    else:
      print("Não foi possível encontrar uma solução")

if __name__=="__main__":
    n = int(input("Digite: \n[1] Para escolher uma instancia de 1 a 10 para executar.\n[2] Para digitar um nome de arquivo de instancia personalizado\n"))
    if(n==1):
      num = int(input("De 1 a 10, escolha o número da instancia que quer executar: "))
      main(f"fake_news_problem/instances/fn{num}.dat")
    elif(n==2):
      nome = input("Digite o nome completo do arquivo de instancia: ")
      main(nome)
    else:
        print("Opcao INVALIDA!")
                  